# Performance Metrics

This notebook includes a set of utility functions (stolen from kaggle), to calculate performance metrics for the model.



In [1]:
import pandas as pd
import numpy as np

# from: https://www.kaggle.com/cpmpml/faster-metric-computation 

def calculate_overlap(set_pred, set_gt):
    """
    Calculates if the overlap between prediction and
    ground truth is enough fora potential True positive
    """
    # Length of each and intersection
    try:
        len_gt = len(set_gt)
        len_pred = len(set_pred)
        inter = len(set_gt & set_pred)
        overlap_1 = inter / len_gt
        overlap_2 = inter/ len_pred
        return overlap_1 >= 0.5 and overlap_2 >= 0.5
    except:  # at least one of the input is NaN
        return False

def score_feedback_comp_micro(pred_df, gt_df, discourse_type):
    """
    A function that scores for the kaggle
        Student Writing Competition
        
    Uses the steps in the evaluation page here:
        https://www.kaggle.com/c/feedback-prize-2021/overview/evaluation
    """
    gt_df = gt_df.loc[gt_df['discourse_type'] == discourse_type, 
                      ['id', 'predictionstring']].reset_index(drop=True)
    pred_df = pred_df.loc[pred_df['class'] == discourse_type,
                      ['id', 'predictionstring']].reset_index(drop=True)
    pred_df['pred_id'] = pred_df.index
    gt_df['gt_id'] = gt_df.index
    pred_df['predictionstring'] = [set(pred.split(' ')) for pred in pred_df['predictionstring']]
    gt_df['predictionstring'] = [set(pred.split(' ')) for pred in gt_df['predictionstring']]
    
    # Step 1. all ground truths and predictions for a given class are compared.
    joined = pred_df.merge(gt_df,
                           left_on='id',
                           right_on='id',
                           how='outer',
                           suffixes=('_pred','_gt')
                          )
    overlaps = [calculate_overlap(*args) for args in zip(joined.predictionstring_pred, 
                                                     joined.predictionstring_gt)]
    
    # 2. If the overlap between the ground truth and prediction is >= 0.5, 
    # and the overlap between the prediction and the ground truth >= 0.5,
    # the prediction is a match and considered a true positive.
    # If multiple matches exist, the match with the highest pair of overlaps is taken.
    # we don't need to compute the match to compute the score
    TP = joined.loc[overlaps]['gt_id'].nunique()

    # 3. Any unmatched ground truths are false negatives
    # and any unmatched predictions are false positives.
    TPandFP = len(pred_df)
    TPandFN = len(gt_df)
    
    #calc microf1
    my_f1_score = 2*TP / (TPandFP + TPandFN)
    return my_f1_score

def score_feedback_comp(pred_df, gt_df, return_class_scores=False):
    """
    Helper function for model evaluation.
    
    Args:
    pred_df  (pandas.DataFrame): dataframe containing model predictions. Needs to have columns: ['id','class','predictionstring']
    gt_df    (pandas.DataFrame): dataframe of ground truth used for model training
    return_class_scores  (bool): Boolean indicating if we want to return the F1 score for each predicted class.
    
    Returns:
    f1                      (float): F1 score of the model
    (optional) class_scores  (dict): Dictionary of per-class F1 score
    """
    class_scores = {}
    for discourse_type in gt_df.discourse_type.unique():
        class_score = score_feedback_comp_micro(pred_df, gt_df, discourse_type)
        class_scores[discourse_type] = class_score
    f1 = np.mean([v for v in class_scores.values()])
    if return_class_scores:
        return f1, class_scores
    return f1


In [7]:
# Importing the data and scoring 
train_df = pd.read_csv('./data/train.csv')
pred_df = train_df[['id','discourse_type','predictionstring']].copy()
pred_df.columns = ['id','class','predictionstring']

# Here scoring the train data onto itself
score_feedback_comp(pred_df, train_df)

1.0

In [ ]:
# Here scoring a sample of it
pred_df2 = pred_df.sample(frac=0.7).reset_index(drop=True)
score_feedback_comp(pred_df2, train_df)